In [0]:
%pip install warcio
%pip install glob2
%pip install tldextract
%pip install warc3-wet

Python interpreter will be restarted.
Python interpreter will be restarted.
Python interpreter will be restarted.
  Created wheel for glob2: filename=glob2-0.7-py2.py3-none-any.whl size=9321 sha256=576fc709190821b93cc14e27daf581217012f31f9bc6fb7aecb1e43dc4e2d6c5
  Stored in directory: /root/.cache/pip/wheels/c0/dd/ee/ba6164807de7570971e8f160dbe6a4178ff4e5922f48c093be
Successfully built glob2
Python interpreter will be restarted.
Python interpreter will be restarted.
Python interpreter will be restarted.
Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
import gzip
import warcio
import pyspark
import warc
import glob
import glob2
import pandas as pd
import numpy as np
from glob import glob
from warcio.archiveiterator import ArchiveIterator
from urllib.parse import urlparse
import tldextract

In [0]:
#importing paths of wet files from the middle week of each year
wet2013 = glob2.glob('/dbfs/mnt/lsde/datasets/commoncrawl/crawl-data/CC-MAIN-2013-20/segments/1368696381249/wet/*.wet.gz', recursive=True)

wet2014 = glob2.glob('/dbfs/mnt/lsde/datasets/commoncrawl/crawl-data/CC-MAIN-2014-23/segments/1404776400583.60/wet/*.wet.gz', recursive=True)

wet2015 = glob2.glob('/dbfs/mnt/lsde/datasets/commoncrawl/crawl-data/CC-MAIN-2015-22/segments/1432207924799.9/wet/*.wet.gz', recursive=True)

wet2016 = glob2.glob('/dbfs/mnt/lsde/datasets/commoncrawl/crawl-data/CC-MAIN-2016-22/segments/1464049270134.8/wet/*.wet.gz', recursive=True)

wet2017 = glob2.glob('/dbfs/mnt/lsde/datasets/commoncrawl/crawl-data/CC-MAIN-2017-22/segments/1495463605188.47/wet/*.wet.gz', recursive=True)

wet2018 = glob2.glob('/dbfs/mnt/lsde/datasets/commoncrawl/crawl-data/CC-MAIN-2018-22/segments/1526794863277.18/wet/*.wet.gz', recursive=True)

wet2019 = glob2.glob('/dbfs/mnt/lsde/datasets/commoncrawl/crawl-data/CC-MAIN-2019-22/segments/1558232254253.31/wet/*.wet.gz', recursive=True)

wet2020 = glob2.glob('/dbfs/mnt/lsde/datasets/commoncrawl/crawl-data/CC-MAIN-2020-24/segments/1590347385193.5/wet/*.wet.gz', recursive=True)

wet2021 = glob2.glob('/dbfs/mnt/lsde/datasets/commoncrawl/crawl-data/CC-MAIN-2021-21/segments/1620243988696.23/wet/*.wet.gz', recursive=True)

wet2022 = glob2.glob('/dbfs/mnt/lsde/datasets/commoncrawl/crawl-data/CC-MAIN-2022-21/segments/1652662509990.19/wet/*.wet.gz', recursive=True)


In [0]:
#initializing empty lists for counting mentions
count2013 = []
count2014 = []
count2015 = []
count2016 = []
count2017 = []
count2018 = []
count2019 = []
count2020 = []
count2021 = []
count2022 = []

#calling the mention function
count2013 = mention(wet2013)
count2014 = mention(wet2014)
count2015 = mention(wet2015)
count2016 = mention(wet2016)
count2017 = mention(wet2017)
count2018 = mention(wet2018)
count2019 = mention(wet2019)
count2020 = mention(wet2020)
count2021 = mention(wet2021)
count2022 = mention(wet2022)

In [0]:
def mention(wet):
    #initializing the mention counters
    mysql = 0
    postgresql=0
    oracle=0
    duckdb=0
    clickhouse=0
    spark=0
    microsoft=0
    pages=0


    for f in range(min(200,len(wet))): #opening at most 200 WET files
        foo = warc.open(wet[f]) #reading the WET file
        for x in foo: #iterating over the webpages
            pages+=1 #webpage counter
            if x.header.get('warc-target-uri', None): #skipping over webpages with no URI
                txt = x. payload.read().lower() #reading the plaintext and ignoring case sensitivity
                if 'database'.encode('utf-8') in txt: #word sense disambiguation by checking first for a 'database' mention
                    if 'microsoft sql server'.encode('utf-8') in txt: microsoft+=1 #counting the mentions of the database systems
                    if 'mysql'.encode('utf-8') in txt: mysql+=1
                    if 'postgresql'.encode('utf-8') in txt: postgresql+=1
                    if 'oracle'.encode('utf-8') in txt: oracle+=1
                    if 'duckdb'.encode ('utf-8') in txt: duckdb+=1
                    if 'clickhouse'.encode( 'utf-8') in txt: clickhouse+=1
                    if 'spark'.encode('utf-8') in txt: spark+=1
                    
    return [mysql,microsoft,postgresql,oracle,duckdb,clickhouse,spark,pages]

In [0]:
print(count2013)
print(count2014)
print(count2015)
print(count2016)
print(count2017)
print(count2018)
print(count2019)
print(count2020)
print(count2021)
print(count2022)

[12217, 1483, 3684, 11872, 0, 0, 7631, 5445673]
[5271, 864, 1858, 6260, 0, 0, 6050, 3834078]
[17452, 1860, 4043, 15591, 0, 0, 15470, 10294322]
[24477, 2318, 4935, 25737, 0, 0, 19260, 11539265]
[13924, 2030, 3860, 13380, 0, 12, 11726, 9147459]
[12467, 1392, 2280, 9790, 0, 2, 6597, 8570128]
[13236, 1863, 3792, 11318, 0, 56, 8381, 9417413]
[13205, 1860, 3834, 9852, 0, 6, 7867, 9376548]
[9338, 1227, 3709, 7899, 4, 97, 7583, 8215960]
[9920, 1171, 3584, 7293, 14, 37, 6777, 8583116]


In [0]:
df = pd.DataFrame(
    {'databases':['mysql','microsoft','postgresql','oracle','duckdb','clickhouse','spark','pages'],
     '2013': count2013,
     '2014': count2014,
     '2015': count2015,
     '2016': count2016,
     '2017': count2017,
     '2018': count2018,
     '2019': count2019,
     '2020': count2020,
     '2021': count2021,
     '2022': count2022,
    })
df.set_index('databases',inplace=True)

In [0]:
df

,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
databases,,,,,,,,,,
mysql,12217,5271,17452,24477,13924,12467,13236,13205,9338,9920
microsoft,1483,864,1860,2318,2030,1392,1863,1860,1227,1171
postgresql,3684,1858,4043,4935,3860,2280,3792,3834,3709,3584
oracle,11872,6260,15591,25737,13380,9790,11318,9852,7899,7293
duckdb,0,0,0,0,0,0,0,0,4,14
clickhouse,0,0,0,0,12,2,56,6,97,37
spark,7631,6050,15470,19260,11726,6597,8381,7867,7583,6777
pages,5445673,3834078,10294322,11539265,9147459,8570128,9417413,9376548,8215960,8583116
